Total International Trade in Services (excluding travel, transport and banking) by industry 2012-2016

In [1]:
from gssutils import *

if is_interactive():
    import requests
    from cachecontrol import CacheControl
    from cachecontrol.caches.file_cache import FileCache
    from cachecontrol.heuristics import LastModified
    from pathlib import Path

    session = CacheControl(requests.Session(),
                           cache=FileCache('.cache'),
                           heuristic=LastModified())

    sourceFolder = Path('in')
    sourceFolder.mkdir(exist_ok=True)

    inputURL = 'https://www.ons.gov.uk/file?uri=/businessindustryandtrade/internationaltrade/datasets/internationaltradeinservicesreferencetables/alltables2016/internationaltradeinservices2016.xls'
    inputFile = sourceFolder / 'internationaltradeinservices2016.xlsb'
    response = session.get(inputURL)
    with open(inputFile, 'wb') as f:
      f.write(response.content)    

In [2]:
tab = loadxlstabs(inputFile, sheetids='Table C0')[0]

Loading in\internationaltradeinservices2016.xlsb which has size 750080 bytes
Table names: ['Table C0']


In [3]:
observations = tab.excel_ref('D5').expand(DOWN).expand(RIGHT).is_not_blank()

In [4]:
Year = tab.excel_ref('D4').expand(RIGHT).is_not_whitespace()

In [5]:
Flow = tab.excel_ref('E3').expand(RIGHT).is_not_blank()

In [6]:
itis = tab.excel_ref('C5').expand(DOWN) - tab.excel_ref('C20').expand(DOWN)

In [7]:
Dimensions = [
            HDim(Year,'Year',DIRECTLY,ABOVE),
            HDim(itis,'BOP Service',DIRECTLY,LEFT),
            HDim(Flow, 'Flow',CLOSEST,LEFT),
            HDimConst('Measure Type', 'GBP Total'),
            HDimConst('Unit','gbp-million')
            ]

In [8]:
c1 = ConversionSegment(observations, Dimensions, processTIMEUNIT=True)
if is_interactive():
    savepreviewhtml(c1)

In [9]:
new_table = c1.topandas()

In [10]:
new_table['Year'] = pd.to_numeric(new_table['Year'], errors='coerce').fillna(0)

In [11]:
new_table['Year'] = new_table['Year'].astype(int)

In [12]:
new_table.columns = ['Value' if x=='OBS' else x for x in new_table.columns]

In [13]:
new_table['ONS Partner Geography'] = 'Whole world'

In [14]:
new_table['Flow'] = new_table['Flow'].str.lower()

In [15]:
new_table = new_table[new_table['Value'] != '' ]

In [16]:
new_table['International Trade Basis'] = 'BOP'

In [17]:
new_table['BOP Service'] = new_table['BOP Service'].map(
    lambda x: {
        'Section A-B, D-E Other Non Services':'Other Non Services',
        'Section C Manufacturing':'Manufacturing Services',
        'Section F Construction':'Construction Services',
        'Section G Wholesale/Retail Trade': 'Wholesale or Retail Trade Services',
        'Section H Transportation & Storage':'Transportation and Storage Services',
        'Section I Accommodation & Food Services': 'Accommodation and Food Services',
        'Section J Information & Communication': 'Information and Communication Services',
        'Section K Financial & Insurance Activities': 'Financial and Insurance Activities',
        'Section L Real Estate Activities':'Real Estate Activities',
        'Section M Professional, Scientific & Technical Activities':'Professional, Scientific and Technical Activities',
        'Section N Administrative & Support Service Activities':'Administrative and Support Service Activities',
        'Section O-Q Public Administration, Education & Health': 'Public Administration, Education and Health Services',
        'Section R&S Arts, Entertainment & Recreation & Other Service Activities': 'R and S Arts, Entertainment and Recreation and Other Service Activities'

        }.get(x, x))

In [18]:
def user_perc(x):    
    if x.strip() == '':
        return 'Total International Trade in Services'
    else:
        return x   
new_table['BOP Service'] = new_table.apply(lambda row: user_perc(row['BOP Service']), axis = 1)


In [19]:
new_table['Flow'].fillna('All', inplace = True)

In [20]:
def user_perc2(x,y):
    
    if ((str(x) ==  '2012') & (str(y) == 'balance')) : 
        return 'balance'
    elif ((str(x) ==  '2012') & (str(y) == 'exports')) :
        return 'imports'
    elif ((str(x) ==  '2012') & (str(y) == 'All' )) :
        return 'exports'
    else :
        return y
    
new_table['Flow'] = new_table.apply(lambda row: user_perc2(row['Year'], row['Flow']), axis = 1)


In [21]:
new_table = new_table[['ONS Partner Geography', 'Year','Flow','BOP Service', 'International Trade Basis', 'Measure Type','Value','Unit' ]]

In [22]:
# if is_interactive():
#     SubstancetinationFolder = Path('out')
#     SubstancetinationFolder.mkdir(exist_ok=True, parents=True)
#     new_table.to_csv(SubstancetinationFolder / ('tableC0.csv'), index = False)

In [23]:
new_table.head()

,ONS Partner Geography,Year,Flow,BOP Service,International Trade Basis,Measure Type,Value,Unit
0,Whole world,2012,exports,Other Non Services,BOP,GBP Total,1209.55,gbp-million
1,Whole world,2013,exports,Other Non Services,BOP,GBP Total,1375,gbp-million
2,Whole world,2014,exports,Other Non Services,BOP,GBP Total,1568,gbp-million
3,Whole world,2015,exports,Other Non Services,BOP,GBP Total,1417.37,gbp-million
4,Whole world,2016,exports,Other Non Services,BOP,GBP Total,1398,gbp-million
